In [6]:
import copy
import easydict

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim

from skimage.exposure import exposure
from skimage.feature import hog

import numpy as np
import matplotlib.pyplot as plt


# PreParing Data

In [10]:
# Data transforms
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.255]

data_transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        transforms.Grayscale(num_output_channels=1)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        transforms.Grayscale(num_output_channels=1)
    ])
}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [11]:
# Loading Datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transform['train'], download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=data_transform['val'], download=True)

# Defining class names
class_names = train_dataset.classes
print(f'Class names are {class_names}')

# Creating DataLoaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False, num_workers=2)
print('DataLoaders Are Ready.')

Files already downloaded and verified
Files already downloaded and verified
Class names are ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
DataLoaders Are Ready.


# HOG Feature Extraction

In [3]:
# HOG Parameters
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
block_norm = 'L2-Hys'


class HOGFeatureExtractor(nn.Module):
    def __init__(self):
        super(HOGFeatureExtractor, self).__init__()

    def forward(self, x):
        hog_features_list = []
        # Iterating over data in each batch and extract hog features 
        for image in x:
            # Convert Image to numpy for hog
            np_image = image.numpy().squeeze()
            hog_features = hog(
                np_image,
                orientations=orientations,
                pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block,
                block_norm=block_norm
            )
            hog_features_list.append(hog_features)
        return torch.tensor(hog_features_list, dtype=torch.float32)


# Setup pretrained model

In [5]:
# Load pretrained model
pretrained_model = models.resnet18(pretrained=True)

# Freeze all trainable layers
for param in pretrained_model.parameters():
    param.requires_grad = False

# Modifying last classification layer for our dataset
num_features = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_features, 10)

C:\Users\Mahdiar\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Mahdiar\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Mahdiar/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:11<00:00, 4.07MB/s]


In [ ]:
# Defining loss function  and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.fc.parameters(), lr=0.001)

# Train Function 

In [ ]:
hog_features_extractor = HOGFeatureExtractor()

acc_list = easydict.EasyDict({'train': [], 'val': []})
loss_list = easydict.EasyDict({'train': [], 'val': []})


def train_model(model, criterion, optimizer, epoch_num=25):
    # Copy the best model weights for loading at the End
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0

    # Iterating over epochs
    for epoch in range(1, epoch_num + 1):
        print(f'Epoch {epoch}/{epoch_num}:')
        
        # Each epoch has two phase Train and Validation
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            # For calculating Loss and Accuracy at the end of epoch
            running_loss = 0.0
            running_corrects = 0.0
            
            # Iterating over data for training and validation
            for idx, data in enumerate(train_loader, 0):
                inputs, labels = data
                
                # Transfer data and labels to Cuda if is available
                inputs = inputs.to(device)
                labels = labels.to(device)
                 
                # Extract HOG features
                hog_features = hog_features_extractor(inputs)
    
                # Forward Pass
                
                # Show Details
                running_loss += loss.item()
                if i % 